## SRJ

## Easier way of calculating Ham for FeS

In [1]:
import pennylane as qml
import numpy as np

import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

symbols = ["Fe", "S"]
r_bohr = 1.8897259886 
geometry = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.826*r_bohr]])

mol = qml.qchem.Molecule(
    symbols,
    geometry,
    mult = 5,
    basis_name="ANO-RCC-MB",  # Specify the basis set name                                                                                           
    load_data=True        # Crucially, set this flag to True                                                                                          
)

# 3. Pass the Molecule object to molecular_hamiltonian                                                                                                
H, n_qubits = qml.qchem.molecular_hamiltonian(mol, method="openfermion", active_electrons=6, active_orbitals=6)
print(H, n_qubits)

-1654.1650026904947 * I(0) + 0.15128199937129344 * Z(0) + -3.0866452470825644e-08 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -3.0866452470825644e-08 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.02166525041128354 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10)) + -0.02166525041128354 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10)) + 0.15128199937129347 * Z(1) + -3.0866452470825644e-08 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + -3.0866452470825644e-08 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + -0.02166525041128353 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11)) + -0.02166525041128353 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11)) + 0.1405620351111539 * Z(2) + 0.14056203511115384 * Z(3) + 0.14056203511114884 * Z(4) + 0.14056203511114884 * Z(5) + 0.11830009062263815 * Z(6) + 0.1183000906226387 * Z(7) + 0.11830008976942097

In [2]:
from pennylane import qchem

qubits = 12
active_electrons = 6
hf_state = qchem.hf_state(active_electrons, qubits)
dev = qml.device("lightning.qubit", wires = qubits)
@qml.qnode(dev)
def e():
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H)

fe = e()
print('fe', fe)

fe -1655.1633906484171


In [3]:




#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
#warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 12
    active_electrons = active_electrons
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 6


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=50) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using active space, check if you change the H accordingly
HF state is -1655.1633906484171
Total excitations are 117
The adapt iteration now is 0


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.1081249697315093 for operator a⁺(0) a⁺(1) a(10) a(11)
Highest gradient excitation is [0, 1, 10, 11]
Final updated parameters: [0.2342113]
Final cost: -1655.1697677409277
 0: ──X─╭FermionicDoubleExcitation(0.23)─┤  State
 1: ──X─├FermionicDoubleExcitation(0.23)─┤  State
 2: ──X─│────────────────────────────────┤  State
 3: ──X─│────────────────────────────────┤  State
 4: ──X─│────────────────────────────────┤  State
 5: ──X─│────────────────────────────────┤  State
10: ────├FermionicDoubleExcitation(0.23)─┤  State
11: ────╰FermionicDoubleExcitation(0.23)─┤  State
The adapt iteration now is 1


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.06195219704941109 for operator a⁺(4) a⁺(5) a(6) a(7)
Highest gradient excitation is [4, 5, 6, 7]
Final updated parameters: [0.23898768 1.30181338]
Final cost: -1655.1932477128362
 0: ──X─╭FermionicDoubleExcitation(0.24)──────────────────────────────────┤  State
 1: ──X─├FermionicDoubleExcitation(0.24)──────────────────────────────────┤  State
 2: ──X─│─────────────────────────────────────────────────────────────────┤  State
 3: ──X─│─────────────────────────────────────────────────────────────────┤  State
 4: ──X─│────────────────────────────────╭FermionicDoubleExcitation(1.30)─┤  State
 5: ──X─│────────────────────────────────├FermionicDoubleExcitation(1.30)─┤  State
 6: ────│────────────────────────────────├FermionicDoubleExcitation(1.30)─┤  State
 7: ────│────────────────────────────────╰FermionicDoubleExcitation(1.30)─┤  State
10: ────├FermionicDoubleExcitation(0.24)──────────────────────────────────┤  State
11: ────╰FermionicDoubleExcitation(0.24)──

...

The highest operator value is 2.6183953254427678e-05 for operator a⁺(1) a(11)
Highest gradient excitation is [1, 11]
Final updated parameters: [ 3.06108565e-01 -5.99293638e-04 -3.39869799e-05  6.12604741e-01
  6.18191568e-01 -1.15969700e-01 -1.56914008e+00 -4.22088832e-04
 -1.18529794e-01 -3.14177451e+00 -1.57447357e-03  5.96292450e-04
  5.25756617e-04  3.93997246e-04 -1.27518708e-04 -2.66134813e-05
 -3.54658638e-06 -5.39048035e-18 -1.42670350e-17 -3.59614438e-17
 -3.64166690e-17 -3.66442815e-17 -3.66442815e-17 -3.59614438e-17
 -2.77677268e-17 -3.66442815e-17 -3.68718941e-17 -1.95740098e-17
 -3.73271192e-17 -1.82090048e-18 -2.27612560e-19 -1.59328792e-18
 -1.59328792e-18 -1.59328792e-18 -9.10450240e-19 -4.55225120e-19
 -4.55225120e-19 -4.55225120e-19 -4.55225120e-19 -9.10450240e-19
 -9.10450240e-19 -4.55225120e-19 -9.10450240e-19 -9.10450240e-19
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
Final cost: -1655.3085207315082
 0: ──X─╭FermionicDoubleExcitation(0.31)────

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 2.6183953254426593e-05 for operator a⁺(1) a(11)
Highest gradient excitation is [1, 11]
Final updated parameters: [ 3.06108565e-01 -5.99293638e-04 -3.39869799e-05  6.12604741e-01
  6.18191568e-01 -1.15969700e-01 -1.56914008e+00 -4.22088832e-04
 -1.18529794e-01 -3.14177451e+00 -1.57447357e-03  5.96292450e-04
  5.25756617e-04  3.93997246e-04 -1.27518708e-04 -2.66134813e-05
 -3.54658638e-06 -5.39048035e-18 -1.42670350e-17 -3.59614438e-17
 -3.64166690e-17 -3.66442815e-17 -3.66442815e-17 -3.59614438e-17
 -2.77677268e-17 -3.66442815e-17 -3.68718941e-17 -1.95740098e-17
 -3.73271192e-17 -1.82090048e-18 -2.27612560e-19 -1.59328792e-18
 -1.59328792e-18 -1.59328792e-18 -9.10450240e-19 -4.55225120e-19
 -4.55225120e-19 -4.55225120e-19 -4.55225120e-19 -9.10450240e-19
 -9.10450240e-19 -4.55225120e-19 -9.10450240e-19 -9.10450240e-19
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
Final cost: -1655.3085207315066
 0: ──X─╭FermionicDoubleExc

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 2.6183953254411767e-05 for operator a⁺(1) a(11)
Highest gradient excitation is [1, 11]
Final updated parameters: [ 3.06108565e-01 -5.99293638e-04 -3.39869799e-05  6.12604741e-01
  6.18191568e-01 -1.15969700e-01 -1.56914008e+00 -4.22088832e-04
 -1.18529794e-01 -3.14177451e+00 -1.57447357e-03  5.96292450e-04
  5.25756617e-04  3.93997246e-04 -1.27518708e-04 -2.66134813e-05
 -3.54658638e-06 -5.39048035e-18 -1.42670350e-17 -3.59614438e-17
 -3.64166690e-17 -3.66442815e-17 -3.66442815e-17 -3.59614438e-17
 -2.77677268e-17 -3.66442815e-17 -3.68718941e-17 -1.95740098e-17
 -3.73271192e-17 -1.82090048e-18 -2.27612560e-19 -1.59328792e-18
 -1.59328792e-18 -1.59328792e-18 -9.10450240e-19 -4.55225120e-19
 -4.55225120e-19 -4.55225120e-19 -4.55225120e-19 -9.10450240e-19
 -9.10450240e-19 -4.55225120e-19 -9.10450240e-19 -9.10450240e-19
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
Final cost: -1655.3085207315055
 0: ──X─╭Fe

In [4]:
energies

[-1655.1697677409277,
 -1655.1932477128362,
 -1655.2057004240974,
 -1655.2432084730522,
 -1655.2465865489985,
 -1655.2473068410845,
 -1655.2770116033885,
 -1655.2772736778743,
 -1655.277588848754,
 -1655.3085201565204,
 -1655.308520218471,
 -1655.3085204282484,
 -1655.3085204490428,
 -1655.3085205859693,
 -1655.3085207256174,
 -1655.3085207256797,
 -1655.308520731544,
 -1655.3085207315426,
 -1655.3085207315423,
 -1655.3085207315416,
 -1655.3085207315403,
 -1655.308520731539,
 -1655.308520731538,
 -1655.3085207315366,
 -1655.3085207315353,
 -1655.308520731534,
 -1655.308520731533,
 -1655.3085207315319,
 -1655.3085207315312,
 -1655.3085207315298,
 -1655.3085207315287,
 -1655.308520731527,
 -1655.3085207315262,
 -1655.3085207315248,
 -1655.3085207315237,
 -1655.3085207315225,
 -1655.3085207315212,
 -1655.3085207315198,
 -1655.3085207315187,
 -1655.308520731517,
 -1655.3085207315157,
 -1655.3085207315148,
 -1655.3085207315132,
 -1655.3085207315132,
 -1655.308520731512,
 -1655.3085207315112

In [6]:
ash_excitation

[[0, 1, 10, 11],
 [4, 5, 6, 7],
 [2, 3, 8, 9],
 [2, 4, 6, 8],
 [3, 5, 7, 9],
 [0, 10],
 [2, 5, 7, 8],
 [3, 4, 6, 9],
 [1, 11],
 [2, 5, 6, 9],
 [0, 10],
 [0, 1, 10, 11],
 [0, 1, 10, 11],
 [0, 1, 10, 11],
 [0, 10],
 [0, 1, 10, 11],
 [0, 1, 10, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11],
 [1, 11]]

In [7]:
old_grad

[0.1081249697315093,
 0.06195219704941109,
 0.061952196243247507,
 0.02105367382956475,
 0.052481668206331905,
 0.01843440378364843,
 0.01193063370169216,
 0.014494616848028573,
 0.011107165857812178,
 0.0068683705272062434,
 0.00027783112703850914,
 0.00030314419694977353,
 0.0001644339855340256,
 9.814716152777746e-05,
 0.0001719711750879589,
 5.7252633420954976e-05,
 6.554073348143794e-05,
 2.6183953254195658e-05,
 2.618395325420043e-05,
 2.618395325424686e-05,
 2.6183953254240734e-05,
 2.6183953254240056e-05,
 2.6183953254250356e-05,
 2.618395325426006e-05,
 2.618395325427025e-05,
 2.6183953254276296e-05,
 2.618395325427028e-05,
 2.6183953254271905e-05,
 2.618395325427822e-05,
 2.6183953254301178e-05,
 2.618395325431129e-05,
 2.6183953254333677e-05,
 2.6183953254331346e-05,
 2.6183953254324895e-05,
 2.6183953254324787e-05,
 2.6183953254328066e-05,
 2.6183953254327822e-05,
 2.618395325432224e-05,
 2.6183953254356554e-05,
 2.618395325434815e-05,
 2.61839532543404e-05,
 2.618395325433